<a href="https://colab.research.google.com/github/MindsMend-org/Biofeedback-Mindsync/blob/main/VoiceCloned_PDF_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Voice Cloning Setup { display-mode: "code" }
# Cloning the repository
!git clone https://github.com/misbah4064/Real-Time-Voice-Cloning-1.git
%cd Real-Time-Voice-Cloning-1/
# Installing the dependencies
!pip install -q -r requirements.txt
!apt-get install -qq libportaudio2
!pip install pyttsx3
!pip install PyPDF2
!pip install soundfile
# Downloading pretrained data and unzipping it
!gdown https://drive.google.com/uc?id=1z_QgssuHaeyqv60H7tKkFavS7nzQosIl
!unzip pretrained_models.zip
# Initializing all the encoder libraries
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import PyPDF2
import pyttsx3
from google.colab import files
encoder_weights = Path("pretrained_models/encoder/saved_models/pretrained.pt")
vocoder_weights = Path("pretrained_models/vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("pretrained_models/synthesizer/saved_models/pretrained/pretrained.pt")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)
#synthesizing audio
from IPython.display import clear_output
clear_output()

def readText(text):
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  generated_wav = encoder.preprocess_wav(generated_wav)
  return [generated_wav, synthesizer.sample_rate]
# option = "Default-Elon Musk" #@param ["Default-Elon Musk", "Upload"]
# #filename = "elon audio.wav" #@param {type: "string"}
# if option=="Default-Elon Musk":
#   in_fpath = Path("pretrained_models/elon audio.wav")
# else:
#   from google.colab import files
#   uploaded = files.upload()
#   filepath = next(iter(uploaded))
#   in_fpath = Path(filepath)

# reprocessed_wav = encoder.preprocess_wav(in_fpath)
# original_wav, sampling_rate = librosa.load(in_fpath)
# preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
# embed = encoder.embed_utterance(preprocessed_wav)

In [ ]:
Cloning_Source = "Upload" #@param ["Default-Elon Musk", "Upload"]

if Cloning_Source=="Default-Elon Musk":
  in_fpath = Path("pretrained_models/elon audio.wav")
else:
  from google.colab import files
  uploaded = files.upload()
  filepath = next(iter(uploaded))
  in_fpath = Path(filepath)

reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)

print("Upload Audio Book")
uploaded_book = files.upload()
filepath = next(iter(uploaded_book))
bookPath = Path(filepath)
path = open(bookPath, 'rb')
pdfReader = PyPDF2.PdfFileReader(path)
totalPages = pdfReader.numPages
clear_output()


In [ ]:
Pages_to_Read = "Custom" #@param ["All", "Custom"]
if Pages_to_Read=="Custom":
  from_page_number = 14 #@param {type:"integer"}
  to_page_number = 14 #@param {type:"integer"}
error = 0
if Pages_to_Read=="All":
  from_page_number = 0
  to_page_number = totalPages-1

if to_page_number > totalPages-1:
  print("Error: You are selecting pages that don't exist in the PDF")
  error = 1
if from_page_number<0:
  print("Error: There are no negative page numbers")
  error = 1
if from_page_number>to_page_number:
  print("Please select from_page_number less than to_page_number")
  error = 1

import soundfile
init = 0
text = ""
if error ==0:
  for page in range(from_page_number, to_page_number+1):
    from_page = pdfReader.getPage(page)
    text = from_page.extractText()
    lines = text.split('\n')
    for line in lines:
      generated_wav, rate = readText(line)
      # display(Audio(generated_wav, rate=synthesizer.sample_rate))
      if init==0:
        init = 1
        # data, rate = soundfile.read('testing.wav')
        soundfile.write('generated_audio.wav', generated_wav, rate)
        # soundfile.SoundFile.close()

      else:
        with soundfile.SoundFile('generated_audio.wav', 'r+') as wfile:
            wfile.seek(0,soundfile.SEEK_END)
            wfile.write(generated_wav)
    # print(lines[:2])
  # readText(text)

  # print(text)
